In [ ]:
import pandas as pd
import numpy as np
import string
from pprint import pprint
import itertools

# Filter to current df

In [ ]:
# TODO: make sure same people as last time aren't matched
# TODO: fill in previous values

df = pd.read_csv('data/responses/responses_8_28.csv')

In [ ]:
df.columns

In [ ]:
# this_week 
df = df[df['Timestamp'] > '2020/08/22']
df.groupby('What kind of interaction are you after this week?').count()['Name']

In [ ]:
print(len(df))
all_matches = []

# Load in previous matches

In [ ]:
prev_matches = pd.read_csv('data/matches/matched_pairs_8_7.csv')
prev_matches = prev_matches[['Email 1', 'Email 2', 'Email 3 (if applicable)']]
prev_matches = prev_matches.values.tolist()

prev_pairs = []
for m in prev_matches:
    for combo in itertools.combinations(m, 2):
        assert(len(combo) == 2)
        if (type(combo[0]) == float and np.isnan(combo[0])) or (type(combo[1]) == float and np.isnan(combo[1])):
            continue
        prev_pairs.append(combo)

# Mentorship Matching (supplemented w/ Random)

In [ ]:
mentorship = df[df['What kind of interaction are you after this week?'] == 'PhD mentorship']
random = df[df['What kind of interaction are you after this week?'].isin(['Random', 'Random/Other'])]

In [ ]:
pd.set_option('display.max_columns', 500)

# Friendship Matching

In [ ]:
friends = df[df['What kind of interaction are you after this week?'] == 'Friendship outside of work']

In [ ]:
friends.columns

In [ ]:
def get_clean_hobbies(hobs):
    hobby_lookup = {
        'geopolitics': 'politics',
        'foodie': 'food',
        'movie buff': 'movies',
        'painting': 'art',
        'exploring cities': 'traveling',
        'transit': 'traveling',
        'geography': 'history',
        'japanese language': 'languages',
        'gaming': 'video games',
        'playing music': 'music',
        'physical fitness': 'fitness',
        'exercise': 'fitness',
        'swimming': 'fitness',
        'super smash bros melee': 'video games',
        'chess': 'board games',
        'want to get involved in volunteering': 'volunteering',
        'wine': 'alcohol',
        'travel': 'traveling',
        'FIRST': ['stem', 'robotics'],
        'musicviolin': 'music',
        'entrepreneurship': 'startups',
        'philosophy': 'humanities',
        'coffee': 'beverages',
        'math': 'stem',
        'roller skating': 'playing sports',
        'reptiles': 'animals',
        'visual art': 'art',
        'economics': 'humanities',
        'tech': 'stem',
        'piano': 'music',
        'card game': 'board games',
        'algorithm': 'stem',
        'learning languages': 'languages',
        'classical music': 'music',
        'reading history': 'history',
        'ice skating': 'playing sports',
        'practicing italian': 'languages',
        'watching tv': 'tv',
        'tv shows': 'tv',
        'breweries': 'alcohol',
        'wineries': 'alcohol',
        'occasional visits adventure sports and theme parks': 'playing sports',
        'watching movies': 'movies',
        'singing': 'music',
        'listening to podcasts and music': 'podcasts/audiobooks',
        'diy handyman': 'diy',
        'backpacking': 'outdoors',
        'outdoor adventures': 'outdoors',
        'guitar': 'music',
        'surfing': 'playing sports',
        'general banter': 'conversation',
        'procrastinating by reading about procrastination': 'productivity',
        'productivity hacks': 'productivity',
        'cocktail making': 'alcohol',
        'watching avatar on netflix': 'tv',
        'action-adventure': 'movies',
        'moviesmystery': 'movies',
        'lazy cooking': 'cooking',
        'violin': 'music',
        'performance modelling': 'stem',
        'cycling': 'biking',
        'resource management': 'productivity',
        'scheduling': 'productivity',
        'networks': 'stem',
        'and animal facts': 'animals',
        'memes': 'comedy',
        'musical instruments': 'music',
        'social justice': 'volunteering',
        'playing violin': 'music',
        'vegetables': 'food',
        'bread': 'food',
        'tea': 'beverages',
        'fermenting things': 'cooking',
        'policy': 'politics',
        'verification': 'stem',
        'emacs': 'stem',
        'systems': 'stem',
        'photography': 'art',
        'i play drums': 'music',
        'climbing': 'climbing',
        'drawing': 'art',
        'running': 'running',
        'jogging': 'running',
        'learning about pittsburgh': 'pittsburgh',
        'watching sports in quarantine': 'watching sports',
        'ultimate frisbee': 'playing sports',
        'want to start biking': 'biking',
        'politics': 'politics',
        'playing music': 'music',
        'violin': 'music',
        'whiskey': 'alcohol',
        'adventure sports': 'playing sports',
        'stand-up': 'comedy',
        'tv': 'tv',
        'films': 'movies',
        'improv': 'comedy',
        'pets': 'animals',
        'video games': 'video games',
        'sports': 'watching sports',
        'gbbo': ['baking', 'cooking'],
        'npr one': 'podcasts/audiobooks',
        'audiobooks': 'podcasts/audiobooks',
        'rock climbing': 'climbing',
        'badminton': 'playing sports',  # week 8/28
        'mystery': 'mystery',
        'currently watching legend of korra on netflix': 'tv',
        'formula1': 'watching sports',
        'playing tennis': 'playing sports',
        'things to do in pittsburgh': 'pittsburgh',
        'podcasts': 'podcasts/audiobooks',
        'watching musicals': 'music',
        'reading books': 'reading',
        'fiction stories and things about grammar':'reading',
        'eating ice cream':'food',
        'writing':'reading',
        'scrolling through twitter':'twitter',
        'space':'stem',
        'rocks':'stem',
        'roofs':'diy',
        'the great british bake off':['baking', 'cooking'],
        'marvel':'tv',
        'agents of shield':'tv',
        'gym':'fitness',
        'bouldering':'climbing',
        'finding creative ways to exercise in quarantine':'fitness',
        'sci-fi':'sci-fi',
        'basketball': 'playing sports',
        'jazz rnb and foreign music': 'music',
        'books': 'reading',
        'learning about pittsburgh area': 'pittsburgh',
        'dinosaurs': 'animals',
        'cooking and experimenting with different cuisines': 'cooking',
        'taking walks': 'outdoors',
        'general banter and discussions': 'conversation',
        'textiles': 'diy',
        'knitting': 'diy',
        'crocheting': 'diy',
        'sewing': 'diy',
        'mechanical engineering': 'stem',
        'experimental music': 'music',
        'listening to music and podcasts': 'podcasts/audiobooks',
        'white water rafting': 'outdoors',
        'mentoring others': 'volunteering',
        'advocating for social justice': 'volunteering',
        'skiing': 'playing sports',
        'snowboarding': 'playing sports',
        'outdoor activities': 'outdoors',
        'language learning': 'languages',
        'digital art': 'art',
        'christian faith': 'christian',
        'board game': 'board games',
        'go': 'board games',
        'about pittsburgh area': 'pittsburgh',
        'watching legend of korra on netflix': 'tv',
    }
    
    stopwords = [
        'academic',
        'currently',
        'esp',
        'learning',
        'etc',
    ]
    clean = []
    for h in hobs:
        h = h.strip().lower()
        for s in stopwords:
            h = h.replace(s, '')   
            h = h.strip()
        
        if len(h) == 0:
            continue
            
        h = hobby_lookup.get(h, h)
        if type(h) == str:
            clean.append(hobby_lookup.get(h, h))
        else:
            assert(type(h) == list)
            clean += h
        
    clean = [c for c in clean if len(c) > 0]
    clean = set(clean)
    return clean


In [ ]:
friends.groupby('How would you like to be matched?').count()['Name']

In [ ]:
len(friends)

In [ ]:
friends.columns

In [ ]:
time_to_person = {}
hobby_ct = {}
friends = friends[[
    'Username',
    'Name', 
    'What times would you be able to meet? (Pittsburgh time) [Morning]',
    'What times would you be able to meet? (Pittsburgh time) [Afternoon]',
    'What times would you be able to meet? (Pittsburgh time) [Evening]',
    'Where would you want to be able to meet?', 
    'Hobbies/Interests', 
    'How would you like to be matched?',
    'Anything else you want us to know for matching purposes?']]

people = []
for (i, row) in friends.iterrows():
    username, name, m_times, a_times, e_times, where, hobbies, how, other = row
    
    if (m_times is None) or (type(m_times) == float and np.isnan(m_times)):
        m_times = ''
    if (a_times is None) or (type(a_times) == float and np.isnan(a_times)):
        a_times = ''
    if (e_times is None) or (type(e_times) == float and np.isnan(e_times)):
        e_times = ''
    ts = list(['{} Morning'.format(t) for t in m_times.split(';')])
    ts += list(['{} Afternoon'.format(t) for t in a_times.split(';')])
    ts += list(['{} Evening'.format(t) for t in e_times.split(';')])
    wh = [] if (where is None or (type(where) == float and np.isnan(where))) else where.split(';')
    
    if (hobbies is None or (type(hobbies) == float and np.isnan(hobbies))):
        hob = []
    else:
        hobbies = hobbies.replace('\n', ',').replace(':', ',').replace(';', ',')
        hobbies = hobbies.translate(str.maketrans(string.punctuation.replace('-', ''), ',' * (len(string.punctuation) - 1), ''))
        hobbies = hobbies.split(',')
        hob = [h.lower().strip() for h in hobbies]
    oth = '' if (other is None or (type(other) == float and np.isnan(other))) else other
    
    for h in set(get_clean_hobbies(hob)):
        hobby_ct[h] = hobby_ct.get(h, 0) + 1
        
    for t in ts:
        time_to_person[t] = time_to_person.get(t, []) + [(name, wh, hob)]
    people.append([username, name, ts, wh, hob, how, oth])

In [ ]:
def get_match_score(p1, p2):
    reasons = []
    
    usr1, name1, t1, wh1, hob1, how1, oth1 = p1
    usr2, name2, t2, wh2, hob2, how2, oth2 = p2
    
    score = 0
    
    if ((usr1, usr2) in prev_pairs) or ((usr2, usr1) in prev_pairs):
        score -= 5
    
    # based on timing    
    common_times = set(t1).intersection(set(t2))
    if len(common_times) > 0:
        score += 2
        reasons.append(['common time', common_times])
    
    # based on location
    if wh1 is None:
        wh1 = ['Over Zoom', 'In-person in Pittsburgh (physically distant and outside)']
    if wh2 is None:
        wh2 = ['Over Zoom', 'In-person in Pittsburgh (physically distant and outside)']
    common_place = set(wh1).intersection(set(wh2))
    if len(common_place) > 0:
        score += 1
        if len(wh1) == 1 and len(wh2) == 1:
            score += 0.5
        reasons.append(['common place', common_place])
    else:
        score -= 3
    
    # based on hobbies
    hob1 = get_clean_hobbies(hob1)  # in the future it may be better to tokenize
    hob2 = get_clean_hobbies(hob2)
    common_interests = hob1.intersection(hob2)
    reasons.append(['common interests', common_interests])
    for ci in common_interests:
        if hobby_ct[ci] <= 5:
            score += 3
        else:
            score += 2
#     score += 2 * len(common_interests)

    if (how1 == 'In a pair') and (how2 == 'In a pair'):
        score += 2

    return score, reasons

In [ ]:
# first, compute pairwise cores

match_scores = []
for (i, p1) in enumerate(people):
    for (j, p2) in enumerate(people):
        if not (i < j):
            continue
        
        usr1, name1, t1, wh1, hob1, how1, oth1 = p1
        usr2, name2, t2, wh2, hob2, how2, oth2 = p2
            
        s, reasons = get_match_score(p1, p2)
        
        if how1 == 'In a group of 3-4' or how2 == 'In a group of 3-4':
            s -= 5
        if how1 == 'No preference' and how2 == 'No preference':
            s -= 1
        
        match_scores.append([(i, j), s])

In [ ]:
# next compute triple scores for the remaining people

triple_scores = []
for (i, p1) in enumerate(people):
    for (j, p2) in enumerate(people):
        for (k, p3) in enumerate(people):
            if not (i < j and j < k):
                continue

#             if (i in matched_people) or (j in matched_people) or (k in matched_people):
#                 continue
                
            usr1, name1, t1, wh1, hob1, how1, oth1 = p1
            usr2, name2, t2, wh2, hob2, how2, oth2 = p2
            usr2, name3, t3, wh3, hob3, how3, oth3 = p3

#             assert('In a pair' not in set([how1, how2, how3]))
            
            s12, reasons12 = get_match_score(p1, p2)
            s23, reasons23 = get_match_score(p2, p3)
            s13, reasons13 = get_match_score(p1, p3)
            s = (s12 + s23 + s13) / 2.5
            
            if 'In a pair' in set([how1, how2, how3]):
                s -= 6
            
            if 'In a group of 3-4' in set([how1, how2, how3]):
                s += 2
            
            triple_scores.append([(i, j, k), s])

# triple_scores = list(reversed(sorted(triple_scores, key=lambda x: x[1])))
# for idx, s in triple_scores:
#     i, j, k = idx
#     if (i in matched_people) or (j in matched_people) or (k in matched_people):
#         continue
    
#     p1 = people[i]
#     p2 = people[j]
#     p3 = people[k]
    
#     matches.append((i, j, k))
#     matched_people.add(i)
#     matched_people.add(j)
#     matched_people.add(k)
        
#     print('------------------------')
#     print('score: ', s)
#     print('P1: ', p1[1])
#     print('hobbies: ', p1[4])
#     print('how: ', p1[5])
#     print('other: ', p1[6])
#     print('')
#     print('P2: ', p2[1])
#     print('hobbies: ', p2[4])
#     print('how: ', p2[5])
#     print('other: ', p2[6])
#     print('')
#     print('P3: ', p3[1])
#     print('hobbies: ', p3[4])
#     print('how: ', p3[5])
#     print('other: ', p3[6])
#     print('')
#     pprint(get_match_score(p1, p2)[1])


In [ ]:
# all cleaned tokens
list(reversed(sorted(hobby_ct.items(), key=lambda x: x[1])))

In [ ]:
all_friend_scores = match_scores + triple_scores
all_friend_scores = list(reversed(sorted(all_friend_scores, key=lambda x: x[1])))
all_friend_scores[:2]

In [ ]:
# PRINT FINAL MATCHES

matches = []
matched_people = set([])
for idx, s in all_friend_scores:
    assert(len(idx) in [2, 3])
    
    if len(matched_people.intersection(set(list(idx)))) > 0:
        continue
        
    st = '{}\nscore: {}\n\n'.format(idx, s)
    for (_, i) in enumerate(list(idx)):
        matched_people.add(i)
        p = people[i]
        st += 'P{}: {}\nhobbies: {}\nhow: {}\nother: {}\n\n'.format(
            _, p[1], p[4], p[5], p[6])
    print(st)
    matches.append(idx)
    if len(idx) == 2:
        pprint(get_match_score(people[idx[0]], people[idx[1]])[1])
    print('------------------------')

In [ ]:
print('Remaining unmatched friends: ', list([i for i in range(len(people)) if i not in matched_people]))

In [ ]:
e1 = []
e2 = []
e3 = []
for idxs in matches:
    e1.append(people[idxs[0]][0])
    e2.append(people[idxs[1]][0])
    if len(idxs) == 3:
        e3.append(people[idxs[2]][0])
    else:
        e3.append(np.nan)
    
print(e1)
print(e2)
print(e3)
friend_emails = pd.DataFrame({
    'Email 1': e1,
    'Email 2': e2,
    'Email 3 (if applicable)': e3
})

# Review

In [ ]:
# read in the matched pairs and look for unmatched
ms = pd.read_csv('data/matches/manual_matches_8_29.csv')[['Email 1', 'Email 2', 'Email 3 (if applicable)']]
ms = pd.concat([ms, friend_emails], axis=0)
ms = ms.reset_index()
ms

In [ ]:
# look for any missing
full = df['Username'].tolist()
done = ms['Email 1'].tolist() + \
       ms['Email 2'].tolist() + \
       ms['Email 3 (if applicable)'].tolist()
done = [i for i in done if type(i) == str]

incomplete = []
for i in full:
    if (i not in done):
        incomplete.append(i)
        
incomplete

In [ ]:
pd.set_option('max_colwidth', 100)
incomplete = df[df['Username'].isin(incomplete)]
incomplete

In [ ]:
ms.to_csv('data/matches/matched_pairs_8_29.csv', index=False)

In [ ]:
full.columns

In [ ]:
ms = pd.read_csv('data/matches/matched_pairs_8_29.csv')
full = pd.read_csv('data/responses/responses_8_28.csv')

d = {}
for (i, row) in full.iterrows():
    dd = dict(zip(full.columns.tolist(), row.tolist()))
    m_times = dd['What times would you be able to meet? (Pittsburgh time) [Morning]']
    a_times = dd['What times would you be able to meet? (Pittsburgh time) [Afternoon]']
    e_times = dd['What times would you be able to meet? (Pittsburgh time) [Evening]']
    if (m_times is None) or (type(m_times) == float and np.isnan(m_times)):
        m_times = ''
    if (a_times is None) or (type(a_times) == float and np.isnan(a_times)):
        a_times = ''
    if (e_times is None) or (type(e_times) == float and np.isnan(e_times)):
        e_times = ''
    ts = list(['{} Morning'.format(t) for t in m_times.split(';')])
    ts += list(['{} Afternoon'.format(t) for t in a_times.split(';')])
    ts += list(['{} Evening'.format(t) for t in e_times.split(';')])
    ts = ';'.join(ts)

    
    d[dd['Username']] = {
        'name': dd['Name'],
        'times': ts,
        'location': dd['Where would you want to be able to meet?'],
        'interaction': dd['What kind of interaction are you after this week?'],
        'hobbies': dd['Hobbies/Interests'],
        'research': dd['Research topics/interests'],
        'mentor/mentee': dd['Would you like to be a mentor and/or mentee?'],
        'background': dd['Background'],
        'other': [dd['Anything else you want us to know for matching purposes?'],
                  dd['Anything else you want us to know for matching purposes?.1'],
                  dd['Anything else you want us to know for matching purposes?.2'],
                  dd['Anything else you want us to know for matching purposes?.3']],
    }

In [ ]:
groups = []
N = 0

problems = []
for (i, row) in ms.iterrows():
    print('\n\n=============================================================\n\n')
    _,email1, email2, email3 = row.tolist()
    
    name1 = d[email1]['name'].split(' ')[0]
    name2 = d[email2]['name'].split(' ')[0]
    if type(email3) == str:
        name3 = d[email3]['name'].split(' ')[0]
    else:
        name3 = None

    t1 = set(d[email1]['times'].split(';'))
    t2 = set(d[email2]['times'].split(';'))
    time = t1.intersection(t2)
    if type(email3) == str:
        t3 = set(d[email3]['times'].split(';'))
        time = time.intersection(t3)
    if len(time) == 0:
        time = None
        print('NO TIME')
        print(email1, email2)
        problems.append(email1)
        problems.append(email2)
    else:
        time = ', '.join(list(time))

    l1 = set(d[email1]['location'].split(';'))
    l2 = set(d[email2]['location'].split(';'))
    loc = l1.intersection(l2)
    if type(email3) == str:
        l3 = set(d[email3]['location'].split(';'))
        loc = loc.intersection(l3)
    if len(loc) == 0:
        loc = None
        print('NO LOCATION')
        print(email1, email2)
        problems.append(email1)
        problems.append(email2)
    else:
        location = ', '.join(list(loc))
    
    interactions = []
    interactions.append(d[email1]['interaction'])
    interactions.append(d[email2]['interaction'])
    
    

In [ ]:
problems

In [ ]:
pd.set_option('max_colwidth', 500)
full[full['Username'].isin(problems)]

In [ ]:
df.columns

In [ ]:
pd.set_option('max_rows', 500)
df[['Username', 
    'Anything else you want us to know for matching purposes?',
    'Anything else you want us to know for matching purposes?.1',
    'Anything else you want us to know for matching purposes?.2',
    'Anything else you want us to know for matching purposes?.3']]

In [ ]:
sum([len([v for v in m if type(v) == str]) for m in ms.values])